In [4]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import datetime
from datetime import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler


%matplotlib inline
plt.style.use('fivethirtyeight')          #Used for replicating graph styles from fivethirtyeight.com

from keras.models import load_model

from binance.client import Client

In [5]:
#Loading the trained model
model = load_model('Bitcoin_model.h5')

#Summarize the model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 4, 20)             1760      
_________________________________________________________________
lstm_1 (LSTM)                (None, 40)                9760      
_________________________________________________________________
dense (Dense)                (None, 1)                 41        
Total params: 11,561
Trainable params: 11,561
Non-trainable params: 0
_________________________________________________________________


In [6]:
#loading the data from Binance API
api_key = ""
secret_key = ""
client = Client(aki_key,secret_key)
check = client.get_klines(symbol='BTCUSDT',interval = Client.KLINE_INTERVAL_1MINUTE)

check[499]

[1618090920000,
 '59063.95000000',
 '59073.20000000',
 '59063.94000000',
 '59071.11000000',
 '1.10544400',
 1618090979999,
 '65297.87885649',
 130,
 '0.63313200',
 '37398.13135273',
 '0']

In [7]:
price = np.array([float(check[i][4]) for i in range(500)])
price = price.reshape(500,1)

scaler = StandardScaler()
scaler.fit(price[:374])

price = scaler.transform(price)

In [8]:
index = [496,497,498,499]

candles = scaler.transform(np.array([float(check[i][4]) for i in index]).reshape(-1,1))

model_feed = candles.reshape(1,4,1)

scaler.inverse_transform(model.predict(model_feed)[0])[0]

59474.4

## Trading

In [ ]:
symbol = "BTCUSDT"   #Code of cryptocurrency
quantity = '0.05'    #quantity to trade

order = False
index = [496,497,498,499]

while True:
    price = client.get_recent_trades(symbol=symbol)
    candle = client.get_klines(symbol=symbol,interval=Client.KLINE_INTERVAL_1MINUTE)
    candles = scaler.transform(np.array([float(candle[i][4]) for i in index]).reshape(-1,1))
    model_feed = candles.reshape(1,4,1)
    
    
    if order == False and float(price[len(price)-1]['price']) < float(scaler.inverse_transform(model.predict(model_feed)[0])[0]):
        
        #client.order_market_buy(symbol=symbol,quantity = quantity)
        order = True
        buy_price = client.get_order_book(symbol=symbol)['asks'][0][0]
        print('Buy @Market Price : ',float(buy_price),'Timestamp : ',str(datetime.now()))
        
    elif order == True and float(price[len(price)-1]['price'])-float(buy_price) >=10:
        
        #client.order_market_sell(symbol=symbol, quantity=quantity)
        order = False
        sell_price = client.get_order_book(symbol=symbol)['bids'][0][0]
        print('Sell @Market Price : ',float(sell_price),'Timestamp : ',str(datetime.now()))


Buy @Market Price :  59049.85 Timestamp :  2021-04-11 02:42:59.909109
Sell @Market Price :  59144.58 Timestamp :  2021-04-11 02:43:20.696173
Buy @Market Price :  59129.54 Timestamp :  2021-04-11 02:43:22.634111
Sell @Market Price :  59144.32 Timestamp :  2021-04-11 02:43:32.257294
Buy @Market Price :  59143.61 Timestamp :  2021-04-11 02:43:34.212380
